<a href="https://colab.research.google.com/github/vivek9048/data-science/blob/main/Rag_system_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install  pdfplumber langchain langchain_community langchain_core  langchain_experimental faiss-cpu

In [9]:
!pip install --upgrade langchain
from langchain.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA


In [11]:
loader=PDFPlumberLoader("/content/extended_neural_network_dataset (1).pdf")
docs= loader.load()

In [12]:
text_splitter=SemanticChunker(HuggingFaceEmbeddings())
documents=text_splitter.split_documents(docs)


<ipython-input-12-7a59701c4532>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  text_splitter=SemanticChunker(HuggingFaceEmbeddings())
<ipython-input-12-7a59701c4532>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter=SemanticChunker(HuggingFaceEmbeddings())
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
embedder=HuggingFaceEmbeddings()

<ipython-input-13-a16a47fef565>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder=HuggingFaceEmbeddings()


In [16]:
vector=FAISS.from_documents(documents,embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [18]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 2.7 MB/s eta 0:00:00


In [32]:
from langchain_groq import ChatGroq
llm=ChatGroq(
    groq_api_key="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",
    temperature=0,
    model_name="deepseek-r1-distill-llama-70b"
)

In [33]:
llm.invoke("what is python?")

AIMessage(content='<think>\n\n</think>\n\nPython is a high-level, interpreted programming language that is widely used for various purposes such as web development, scientific computing, data analysis, artificial intelligence, and more. Created in the late 1980s by Guido van Rossum, Python is known for its simplicity, readability, and ease of use, making it a great language for beginners and experts alike.\n\n### Key Features of Python:\n1. **Easy to Learn**: Python has a simple syntax with a focus on readability, often using English-like keywords and indentation to define code blocks.\n2. **Interpreted Language**: Python code is executed line by line, making it easier to write and test code quickly.\n3. **Versatile**: Python can be used for web development, scripting, data analysis, machine learning, automation, and more.\n4. **Cross-Platform**: Python can run on multiple operating systems, including Windows, macOS, and Linux.\n5. **Extensive Libraries**: Python has a vast collection 

In [72]:
# DEFine prompt

prompt ="""
1. use the following piece of context to answer at the end.
2. if you do not knnow the answer ,just say that ""i dont know"but do not make up an answer on your own.\n
3. kepp the answer crisp and limited to 3,4 sentences

context:{context}
Question: {question}
Helpful Answer: """

QA_CHAIN_PROMPT= PromptTemplate.from_template(prompt)

In [73]:
llm_chain=LLMChain(
    llm=llm,
    prompt =QA_CHAIN_PROMPT,
    callbacks= None,
    verbose = True

)

In [77]:
document_prompt=PromptTemplate(
    imput_variables=["page_context","source"],
    template="Context:\ncontent:{page_content}\nsource;{source}"
)

In [75]:
combine_documents_chain=StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None
)

In [76]:
qa=RetrievalQA(combine_documents_chain=combine_documents_chain,
            verbose=True,
            retriever=retriever,
            return_source_documents=True)

In [78]:
user_input="What are attention mechanisms in neural networks?"
response=qa(user_input)["result"]



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. use the following piece of context to answer at the end.
2. if you do not knnow the answer ,just say that ""i dont know"but do not make up an answer on your own.

3. kepp the answer crisp and limited to 3,4 sentences

context:Context;
content:Residual networks introduce skip connections to improve gradient flow and enable training of very deep models. Graph neural networks (GNNs) can model relational data, such as user interactions in a social network. Few-shot learning methods allow neural networks to generalize from a few examples using meta-learning techniques. Self-supervised learning uses unlabelled data for pretraining, often improving performance on vision and language tasks. Neural architecture search automates the design of neural networks, enabling the discovery of high-performing models. Multimodal learning integrates data from multiple sources, such as text and images, usin

In [79]:
print(response)

<think>
Okay, so I need to figure out what attention mechanisms are in neural networks. I remember from the context provided that attention mechanisms allow neural networks to focus on relevant input features. They're crucial in models like transformers and attention-based RNNs. 

Wait, the context mentions that attention mechanisms help the network focus on important parts of the input. So, in models such as transformers, which are used a lot in NLP tasks, attention is a key component. It allows the model to weigh different parts of the input differently, which helps in tasks like translation or text generation.

I think the main idea is that attention mechanisms improve how neural networks process sequential data by letting them concentrate on the most relevant features. This is especially useful in handling long-term dependencies and making the model more efficient by not wasting resources on less important parts of the data.

So, putting it all together, attention mechanisms are te